In [1]:
# Imports
import os
import pandas as pd
from os import listdir
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import pylev

import re

import seaborn as sns
sns.set_style('darkgrid')
sns.color_palette('Spectral')
import matplotlib.pyplot as plt


import requests

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
import glob
# Step 1: get a list of all csv files in target directory
my_dir = "/home/mark/coursework/DNA/data"
filelist = []
filesList = []
user_frame = []
os.chdir( my_dir )

# Step 2: Build up list of files:
for files in glob.glob("*.txt"):
    fileName, fileExtension = os.path.splitext(files)
    filelist.append(fileName) #filename without extension
    filesList.append(files) #filename with extension
    
    

# Step 3: Build up DataFrame:
df = pd.DataFrame()


dfs = [pd.read_table(ijk, sep='\t', 
                   dtype={'rsid':'str', 'chromosome':'object', 'position':'int', 'genotype':'str'}, 
                   comment='#').assign(ID=ijk) \
       for ijk in filesList]

df = pd.concat(dfs, ignore_index=True, axis=0)

In [3]:
df.head()


,rsid,chromosome,position,genotype,ID
0,rs12564807,1,734462,AA,23andme_CA_V.txt
1,rs3131972,1,752721,AG,23andme_CA_V.txt
2,rs148828841,1,760998,CC,23andme_CA_V.txt
3,rs12124819,1,776546,AA,23andme_CA_V.txt
4,rs115093905,1,787173,GG,23andme_CA_V.txt


In [4]:
df.columns

Index(['rsid', 'chromosome', 'position', 'genotype', 'ID'], dtype='object')

In [5]:
df['ID']

0           23andme_CA_V.txt
1           23andme_CA_V.txt
2           23andme_CA_V.txt
3           23andme_CA_V.txt
4           23andme_CA_V.txt
                 ...        
2230922    23andme_MG_v4.txt
2230923    23andme_MG_v4.txt
2230924    23andme_MG_v4.txt
2230925    23andme_MG_v4.txt
2230926    23andme_MG_v4.txt
Name: ID, Length: 2230927, dtype: object

In [8]:
summary = df.groupby(['genotype', 'ID']).size().unstack().fillna(0)
df6 = df[df.chromosome == "6"]
summary = df6[(df6['position'] >= 31971175) & (df6['position'] <= 32109338)]

summary
summary.nunique()

#summary_table.loc[['genotype', 'ID']]

rsid          162
chromosome      1
position      162
genotype       10
ID              3
dtype: int64

In [68]:
summary_table = (
    df
    .groupby('rsid')
    .filter(lambda x : len(x) > 400)
    .groupby(['rsid', 'ID', 'genotype'])
    .size()
    .unstack()
    .fillna(0)
    )
plt.figure(figsize=(30,30))
sns.heatmap(
    summary_table.apply(lambda x : x / x.sum(), axis=0),
    cmap='BuPu',
    square=True,
    cbar_kws = {'fraction' : 0.02}
)
summary_table
#summary_table.loc[['genotype', 'ID']]

ValueError: zero-size array to reduction operation minimum which has no identity

<Figure size 2160x2160 with 0 Axes>

In [67]:
# Pairwise comparison 
pairwise = pd.DataFrame(
    squareform(pdist(summary)),
    columns = summary.index,
    index = summary.index
)
pairwise

# make pairwise distance matrix
pairwise_top = pd.DataFrame(
    squareform(pdist(summary_table, metric='cosine')),
    columns = summary_table.index,
    index = summary_table.index
)

# plot it with seaborn
plt.figure(figsize=(20,20))
sns.heatmap(
    pairwise_top,
    cmap='OrRd',
    linewidth=1
)

NameError: name 'squareform' is not defined

In [ ]:
# Correlation Matrix 
f = plt.figure(figsize=(19, 15))
plt.matshow(df.corr(), fignum=f.number)
plt.xticks(range(df.shape[1]), df.columns, fontsize=10, rotation=45)
plt.yticks(range(df.shape[1]), df.columns, fontsize=10)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=12)
plt.title('Correlation Matrix', fontsize=14); 

In [14]:
summary

,rsid,chromosome,position,genotype,ID
410184,i5900278,6,31973653,AA,23andme_KSU_V3.txt
410185,rs9501393,6,31974849,CC,23andme_KSU_V3.txt
410186,rs62402680,6,31975113,GG,23andme_KSU_V3.txt
410187,rs3130677,6,32005904,TT,23andme_KSU_V3.txt
410188,rs72552743,6,32006258,GG,23andme_KSU_V3.txt
...,...,...,...,...,...
1839525,rs35580488,6,32097421,GG,23andme_MG_v4.txt
1839526,rs169494,6,32097876,GG,23andme_MG_v4.txt
1839527,rs11962353,6,32100077,GG,23andme_MG_v4.txt
1839528,rs9267803,6,32101762,CC,23andme_MG_v4.txt


In [66]:
pd.options.display.max_rows = 999

#summary.groupby(summary.rsid).min()

for i, g in summary.groupby((~((summary == summary.shift(1)).all(1))).cumsum()):
    combined = pd.DataFrame(g)
   # print(g)
combined
def add_column(frame, fn, name):
    new_col = summary.groupby(['rsid', 'ID']).apply(fn).reset_index(name=name)
    return frame.merge(new_col)

def combined(df):
    for i, g in summary.groupby((~((summary == summary.shift(1)).all(1))).cumsum()):
        return g
   # print(g)
    
metric_frame = add_column(summary, combined, 'combined')

TypeError: reset_index() got an unexpected keyword argument 'name'